In [ ]:
import pandas as pd
import pypsa

n = pypsa.Network(snakemake.input["network"])

# Advanced nuclear electricity source design
disaggregate_nuclear = False
if "nuclear heat source" in n.generators.index:
    disaggregate_nuclear = True

# Calculate cost contribution per category based on capital and marginal cost
data = [
    (
        n.generators["p_nom_opt"] * n.generators["capital_cost"]
        + n.generators["marginal_cost"] * n.generators_t["p"].sum()
    ),
    (
        n.stores["capital_cost"] * n.stores["e_nom_opt"]
        + n.stores["marginal_cost"] * n.stores_t["p"].where(lambda x: x > 0).sum()
    ),
    (
        n.links["capital_cost"] * n.links["p_nom_opt"]
        + n.links["marginal_cost"] * n.links_t["p0"].sum()
    ),
]

# Disaggregated nuclear generator (baseload and extra capacity) for optional analysis
if disaggregate_nuclear:
    data.append(
        pd.Series(
            n.links["capital_cost"]["nuclear generator"]
            * n.generators["p_nom_opt"]["nuclear heat source"],
            index=["nuclear generator baseload"],
        )
    )

    data.append(
        pd.Series(
            (
                (
                    n.links["p_nom_opt"]["nuclear generator"]
                    - n.generators["p_nom_opt"]["nuclear heat source"]
                )
                * n.links["capital_cost"]["nuclear generator"]
                + (n.links_t["p0"].sum() * n.links["marginal_cost"])[
                    "nuclear generator"
                ]
            ),
            index=["nuclear generator extra"],
        )
    )

# Convert to pandas series
ds = pd.concat(data)

# Cost relative to total generation
ds /= n.loads_t["p"].sum().sum()

ds["total system cost"] = n.objective / n.loads_t["p"].sum().sum()

ds.index.name = "variable"
ds.name = "value"


if disaggregate_nuclear:
    # When disaggregating nuclear generator cost into baseload + extra
    # do not keep the original nuclear generator cost
    ds.pop("nuclear generator")

# Aggregation into categories
agg_dict = {
    "total system cost": "Total system cost",
    "natural gas": "Natural gas",
    "natural gas with CCS": "Natural gas with CCS",
    "solar": "Solar",
    "wind": "Wind",
    "battery storage": "Battery storage",
    "battery charge": "Battery storage",
    "battery discharge": "Battery storage",
    "load shedding": "Unmet demand",
    "H2 electrolysis": "PGP",
    "H2 cavern storage": "PGP",
    "H2 fuel cell": "PGP",
    "nuclear": "Nuclear",
    "nuclear heat storage": "TES plus extra generator",
    "nuclear heat source": "Advanced nuclear",
    "nuclear generator baseload": "Advanced nuclear",
    "nuclear generator extra": "TES plus extra generator",
}

# Unaccounted-for indices in aggregation
t = set(ds.index) - set(agg_dict.keys())

if t:
    assert t, f"{t} should be empty"

ds = ds.groupby(agg_dict).sum()

df = ds.to_frame()

df.to_csv(snakemake.output["csv"])